## Setup

Load libraries

In [1]:
import pickle
import requests
import rioxarray
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup as bs
from snowmodels.utils import SnotelData, ConvertData, preprocess_set_to_nan, calculate_lagged_vars, calculate_pptwt

## Load Data

### Load Snow Classification Dataset

Let's load the North American Snow Class dataset. You can download the data on [NSIDC](https://nsidc.org/data/nsidc-0768/versions/1). Please download `SnowClass_NA_300m_10.0arcsec_2021_v01.0.nc`. Download the paper: [Matthew Sturm and Glen E. Liston](https://journals.ametsoc.org/view/journals/hydr/22/11/JHM-D-21-0070.1.xml).

In [2]:
snow_class=rioxarray.open_rasterio('../data/SnowClass_NA_300m_10.0arcsec_2021_v01.0.nc')
print(snow_class)

<xarray.DataArray 'SnowClass' (band: 1, y: 32400, x: 61200)> Size: 2GB
[1982880000 values with dtype=uint8]
Coordinates:
  * band     (band) int64 8B 1
  * x        (x) float64 490kB -180.0 -180.0 -180.0 ... -10.01 -10.0 -10.0
  * y        (y) float64 259kB 90.0 90.0 89.99 ... 0.006944 0.004167 0.001389
    crs      int64 8B 0
Attributes: (12/20)
    lat#long_name:      latitude of grid cell center
    lat#standard_name:  latitude
    lat#units:          degrees_north
    lon#long_name:      longitude of grid cell center
    lon#standard_name:  longitude
    lon#units:          degrees_east
    ...                 ...
    long_name:          snow classification
    flag_values:        [1. 2. 3. 4. 5. 6. 7. 8.]
    flag_meanings:      tundra boreal_forest maritime ephemeral prairie monta...
    _FillValue:         0
    scale_factor:       1.0
    add_offset:         0.0


### Load  SNOTEL Metadata

The SNOTEL metadata can be downloaded from [NRCS](https://wcc.sc.egov.usda.gov/nwcc/yearcount?network=sntl&state=&counttype=statelist). We will load (by web scraping) and preprocess this metadata.

Note: this code works fine at the time of compiling. However, the code (for webscrapping) may break in the future if changes are made the to website.

In [3]:
snotel_metadata_link = "https://wcc.sc.egov.usda.gov/nwcc/yearcount?network=sntl&state=&counttype=statelist" # SNOTEL metadata page

# Download (request) the SNOTEL metadata page content
response = requests.get(snotel_metadata_link)
if response.status_code == 200:
    print("Successfully connected to NRSC to download the SNOTEL metadata.")
else:
    raise Exception("Could not connect to NRSC to download the SNOTEL metadata.")

# Parse the HTML content
soup = bs(response.content, 'html.parser')

# Find the headers of the table with the SNOTEL metadata
headers = soup.find_all('th', class_='scanReportHeader blue')

# Extract the table content
if headers:
    metadata = headers[0].find_parent('table')
else:
    raise Exception("Could not find the SNOTEL metadata table using the headers specified. The header probably changed.")

# Read the table content into a a list of pandas dataframe objects

snotel_metadata = pd.read_html(StringIO(str(metadata)))[0]

Successfully connected to NRSC to download the SNOTEL metadata.


In [4]:
snotel_df=(
    snotel_metadata
    .query('ntwk == "SNTL"')
    .assign(
        station_id = lambda x: x.site_name.str.extract(r'\((\d+)\)'),
        snowclass = lambda x: x.loc[:, ["lon", "lat"]].apply(lambda y: ConvertData.get_snow_class(lon=y.lon, lat=y.lat, raster=snow_class).capitalize(), axis=1),
        triplets= lambda x: x.loc[:, ['state', 'station_id', 'ntwk']].apply(lambda y: y.station_id + ':' + y.state + ':' + y.ntwk,axis=1)
    )
    .filter(items=['site_name', 'lat', 'lon', 'elev', 'triplets', 'snowclass'])
    .rename(columns={
        'lat': 'latitude',
        'lon': 'longitude',
        'elev': 'elevation'
    }) 
    .replace({'snowclass': {'Ocean': 'Maritime'}}) ## there are some stations at the Ocean. We will replace it with Maritime
)

snotel_df.to_parquet('../data/snotel_with_class.parquet', compression='gzip', index=False)
snotel_df.info()
snotel_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 911 entries, 0 to 910
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   site_name  911 non-null    object 
 1   latitude   911 non-null    float64
 2   longitude  911 non-null    float64
 3   elevation  911 non-null    int64  
 4   triplets   911 non-null    object 
 5   snowclass  911 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 49.8+ KB


,site_name,latitude,longitude,elevation,triplets,snowclass
0,Alexander Lake (1267),61.75,-150.89,170,1267:AK:SNTL,Alpine
1,American Creek (1189),64.79,-141.23,1020,1189:AK:SNTL,Taiga
2,Anchor River Divide (1062),59.86,-151.31,1650,1062:AK:SNTL,Alpine
3,Anchorage Hillside (1070),61.11,-149.68,1910,1070:AK:SNTL,Tundra
4,Aniak (2065),61.58,-159.58,90,2065:AK:SNTL,Taiga


### Pull data from NRSC using the Metloom API

* Set Metloom API parameters

In [14]:
start_date=(2000, 1, 1)
end_date=(2023, 12, 31)
triplets=snotel_df.triplets.to_list()
station_names=snotel_df.site_name.to_list()
elevation=snotel_df.elevation.to_list()
snow_class=snotel_df.snowclass.to_list()

* Pull the data

In [10]:
snotel_data=SnotelData(
    start_date=start_date,
    end_date=end_date,
    triplets=triplets,
    station_name=station_names,
    elevation=elevation,
    snow_class=snow_class
)

raw_data= snotel_data.grab_daily_data()

## pickle for later use
with open('../data/raw_data.pkl', 'wb') as f:
    pickle.dump(raw_data, f)

In [5]:
## load the pickled data

with open('../data/raw_data.pkl', 'rb') as f:
    raw_data = pickle.load(f)

raw_data.info() # get the info of the raw data
raw_data.head() # get the first 5 rows of the raw data

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 6875223 entries, (Timestamp('2014-10-01 09:00:00+0000', tz='UTC'), '1267:AK:SNTL') to (Timestamp('2023-12-31 08:00:00+0000', tz='UTC'), '878:WY:SNTL')
Data columns (total 17 columns):
 #   Column               Dtype   
---  ------               -----   
 0   geometry             geometry
 1   SWE                  float64 
 2   SWE_units            object  
 3   MIN AIR TEMP         float64 
 4   MIN AIR TEMP_units   object  
 5   MAX AIR TEMP         float64 
 6   MAX AIR TEMP_units   object  
 7   AVG AIR TEMP         float64 
 8   AVG AIR TEMP_units   object  
 9   SNOWDEPTH            float64 
 10  SNOWDEPTH_units      object  
 11  PRECIPITATION        float64 
 12  PRECIPITATION_units  object  
 13  datasource           object  
 14  Elevation            int64   
 15  Station Name         object  
 16  Snow_Class           object  
dtypes: float64(6), geometry(1), int64(1), object(9)
memory usage: 918.1+ MB


,,geometry,SWE,SWE_units,MIN AIR TEMP,MIN AIR TEMP_units,MAX AIR TEMP,MAX AIR TEMP_units,AVG AIR TEMP,AVG AIR TEMP_units,SNOWDEPTH,SNOWDEPTH_units,PRECIPITATION,PRECIPITATION_units,datasource,Elevation,Station Name,Snow_Class
datetime,site,,,,,,,,,,,,,,,,,
2014-10-01 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-02 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-03 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-04 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-05 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,32.72,degF,43.7,degF,37.4,degF,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine


In [3]:
## load the pickled data

with open('../data/raw_data.pkl', 'rb') as f:
    raw_data = pickle.load(f)

raw_data.info() # get the info of the raw data
raw_data.head() # get the first 5 rows of the raw data

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 6875223 entries, (Timestamp('2014-10-01 09:00:00+0000', tz='UTC'), '1267:AK:SNTL') to (Timestamp('2023-12-31 08:00:00+0000', tz='UTC'), '878:WY:SNTL')
Data columns (total 17 columns):
 #   Column               Dtype   
---  ------               -----   
 0   geometry             geometry
 1   SWE                  float64 
 2   SWE_units            object  
 3   MIN AIR TEMP         float64 
 4   MIN AIR TEMP_units   object  
 5   MAX AIR TEMP         float64 
 6   MAX AIR TEMP_units   object  
 7   AVG AIR TEMP         float64 
 8   AVG AIR TEMP_units   object  
 9   SNOWDEPTH            float64 
 10  SNOWDEPTH_units      object  
 11  PRECIPITATION        float64 
 12  PRECIPITATION_units  object  
 13  datasource           object  
 14  Elevation            int64   
 15  Station Name         object  
 16  Snow_Class           object  
dtypes: float64(6), geometry(1), int64(1), object(9)
memory usage: 918.1+ MB


,,geometry,SWE,SWE_units,MIN AIR TEMP,MIN AIR TEMP_units,MAX AIR TEMP,MAX AIR TEMP_units,AVG AIR TEMP,AVG AIR TEMP_units,SNOWDEPTH,SNOWDEPTH_units,PRECIPITATION,PRECIPITATION_units,datasource,Elevation,Station Name,Snow_Class
datetime,site,,,,,,,,,,,,,,,,,
2014-10-01 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-02 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-03 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-04 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,NaN,NaN,NaN,NaN,NaN,NaN,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine
2014-10-05 09:00:00+00:00,1267:AK:SNTL,POINT Z (-150.88967 61.74967 160.00000),0.0,in,32.72,degF,43.7,degF,37.4,degF,0.0,in,0.0,in,NRCS,160,Alexander Lake (1267),Alpine


## Data Cleaning

* Variable Selection and unit conversion.
* Clean prcipitation and temperatures.
* Variable creation:
    - **PPTWT** is the winter (sum of December, January, February) precipitataion per station, per snow season.
    - **TD** is the temperature difference (difference between the mean temperature of the warmest month and the mean temperature of the coldest month).
    - Lagged Temperatures and Precipitation.
* Query `Snow_Depth > 5 and SWE > 3`.
* Query `0.05 <= DENSITY <= 0.6`.
* Rename variables and drop unused ones.
* Create the density function.

### Some Background

At a given point, snow depth ($HS$) is related to Snow Water Equivalent (SWE) by the local bulk density ($\rho_s$):

$$
\text{SWE} = HS \frac{\rho_s}{\rho_w}
$$

where depth is measured in centimeters, density in grams per centimeters cubed, $\rho_w$ is the density of water (1 g cm $^{-3}$), and SWE is measured in centimeters of water. As such,

\begin{align*}
\text{SWE} & = HS \times \frac{\rho_s}{1}\\
& = HS \times \rho_s \\
\rho_s & = \frac{\text{SWE}}{HS}
\end{align*}

### Data Cleaning Stage 1:

In [ ]:
clean_df=(
    raw_data.reset_index()
    .assign(
        Station_Name=lambda x: x["Station Name"].str.extract(r"([^\(]*)")[0].str.strip(),
        SWE=lambda x: ConvertData.inches_to_metric(inches=x.SWE, unit="cm"),
        Snow_Depth=lambda x: ConvertData.inches_to_metric(inches=x.SNOWDEPTH, unit="cm"),
        TAVG=lambda x: ConvertData.fah_to_cel(x["AVG AIR TEMP"]),
        TMIN=lambda x: ConvertData.fah_to_cel(x["MIN AIR TEMP"]),
        TMAX=lambda x: ConvertData.fah_to_cel(x["MAX AIR TEMP"]),
        Elevation=lambda x: ConvertData.feet_to_m(x.Elevation),
        Sturm_DOWY=lambda x: x.datetime.map(lambda y: ConvertData().date_to_DOY(y, algorithm="Sturm", origin=10)),
        Pistochi_DOWY=lambda x: x.datetime.map(lambda y: ConvertData().date_to_DOY(y, algorithm="default", origin=11)),
        DOWY=lambda x: x.datetime.map(lambda y: ConvertData().date_to_DOY(y, algorithm="default", origin=10)),
        month=lambda x: x["datetime"].dt.month,
        year=lambda x: x["datetime"].dt.year,
        winter=lambda x: x["datetime"].map(
            lambda date: (
                f"{date.year - 2}-{date.year - 1}"
                if date.month in [1, 2]
                else f"{date.year - 1}-{date.year}"
            )
        ),
        winter_start=lambda x: x.winter.str.split('-').str[0].astype(int),
        winter_end=lambda x: x.winter.str.split('-').str[1].astype(int)
    )
    .pipe(
        lambda df: df.merge(
            df.groupby('Station Name')['PRECIPITATION'].quantile(0.999).reset_index().rename(columns={'PRECIPITATION': 'threshold'}),
            on='Station Name'
        )
    )
    .pipe(preprocess_set_to_nan)
    .pipe(
        lambda df: df.merge(
            df.groupby(["Station Name", "year", "month"])
            .agg(mean_temp=("AVG AIR TEMP", "mean"))
            .reset_index()
            .groupby(["Station Name", "year"])
            .agg(Temp_Diff=("mean_temp", lambda x: x.max() - x.min()))
            .reset_index(),
            on=["Station Name", "year"],
            how="left",
        )
    )
    .sort_values(by=['Station Name', 'datetime'])
    .set_index('datetime')
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=1)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=2)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=3)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=4)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=5)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=6)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=7)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=8)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=9)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=10)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=11)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=12)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=13)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=14)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=15)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=16)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=17)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=18)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=19)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=20)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=21)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=22)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=23)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=24)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=25)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=26)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=27)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=28)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=29)
    .pipe(calculate_lagged_vars, col_of_interest="TAVG", window=30)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=1)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=2)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=3)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=4)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=5)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=6)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=7)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=8)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=9)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=10)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=11)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=12)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=13)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=14)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=15)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=16)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=17)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=18)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=19)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=20)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=21)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=22)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=23)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=24)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=25)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=26)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=27)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=28)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=29)
    .pipe(calculate_lagged_vars, col_of_interest="PRECIPITATION", window=30)
    .reset_index()
    .rename(columns={"datetime": "Date", "site": "Site"})
    .drop(
        columns=[
            "AVG AIR TEMP",
            "MIN AIR TEMP",
            "MAX AIR TEMP",
            "threshold",
            "geometry",
            "SWE_units",
            "MIN AIR TEMP_units",
            "MAX AIR TEMP_units",
            "AVG AIR TEMP_units",
            "PRECIPITATION_units",
            "SNOWDEPTH_units",
            "datasource",
            "year",
            "Station Name",
            "SNOWDEPTH",
            "month",
            'TAVG',
            'TMIN',
            'TMAX'
        ]
    )
)
clean_df.info()
clean_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6875223 entries, 0 to 6875222
Data columns (total 19 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   Date                   datetime64[ns, UTC]
 1   Site                   object             
 2   SWE                    float64            
 3   PRECIPITATION          float64            
 4   Elevation              float64            
 5   Snow_Class             object             
 6   Station_Name           object             
 7   Snow_Depth             float64            
 8   Sturm_DOWY             float64            
 9   Pistochi_DOWY          int64              
 10  DOWY                   int64              
 11  winter                 object             
 12  winter_start           int64              
 13  winter_end             int64              
 14  Temp_Diff              float64            
 15  TAVG_lag_7d            float64            
 16  TAVG_lag_14d      

,Date,Site,SWE,PRECIPITATION,Elevation,Snow_Class,Station_Name,Snow_Depth,Sturm_DOWY,Pistochi_DOWY,DOWY,winter,winter_start,winter_end,Temp_Diff,TAVG_lag_7d,TAVG_lag_14d,PRECIPITATION_lag_7d,PRECIPITATION_lag_14d
0,2000-01-01 08:00:00+00:00,301:CA:SNTL,9.398,0.0,1886.712,Prairie,Adin Mtn,30.48,1.0,62,93,1998-1999,1998,1999,31.813548,NaN,NaN,NaN,NaN
1,2000-01-02 08:00:00+00:00,301:CA:SNTL,9.398,0.2,1886.712,Prairie,Adin Mtn,35.56,2.0,63,94,1998-1999,1998,1999,31.813548,NaN,NaN,NaN,NaN
2,2000-01-03 08:00:00+00:00,301:CA:SNTL,9.906,0.0,1886.712,Prairie,Adin Mtn,33.02,3.0,64,95,1998-1999,1998,1999,31.813548,NaN,NaN,NaN,NaN
3,2000-01-04 08:00:00+00:00,301:CA:SNTL,9.906,0.1,1886.712,Prairie,Adin Mtn,33.02,4.0,65,96,1998-1999,1998,1999,31.813548,NaN,NaN,NaN,NaN
4,2000-01-05 08:00:00+00:00,301:CA:SNTL,9.906,0.0,1886.712,Prairie,Adin Mtn,35.56,5.0,66,97,1998-1999,1998,1999,31.813548,NaN,NaN,NaN,NaN


### Data Cleaning Stage 2:

In [ ]:
pptwt_df=clean_df.groupby(["Station_Name", "winter"])[['winter_start', 'winter_end', 'PRECIPITATION']].apply(lambda x: calculate_pptwt(group=x, df_of_interest=clean_df)).reset_index(name='PPTWT')

pptwt_df.to_parquet('../data/pptwt_df_updated.parquet', compression='gzip', index=False) # store the pptwt data for each snotel site (it takes a while to calculate).
pptwt_df.info()
pptwt_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19673 entries, 0 to 19672
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station_Name  19673 non-null  object 
 1   winter        19673 non-null  object 
 2   PPTWT         17561 non-null  float64
dtypes: float64(1), object(2)
memory usage: 461.2+ KB


,Station_Name,winter,PPTWT
0,Adin Mtn,1998-1999,NaN
1,Adin Mtn,1999-2000,NaN
2,Adin Mtn,2000-2001,10.4
3,Adin Mtn,2001-2002,16.8
4,Adin Mtn,2002-2003,8.6


In [8]:
clean_df2=(
    clean_df
    .merge(pptwt_df, on=['Station_Name', 'winter'], how='inner')
    .query(
        "Snow_Depth != 0 and SWE != 0 and "
        "Snow_Depth > 5 and SWE > 3"
    )
    .assign(
        Snow_Density=lambda x: x.SWE / x.Snow_Depth,
        TAVG_lag_7d=lambda x: ConvertData.fah_to_cel(x.TAVG_lag_7d),
        TAVG_lag_14d=lambda x: ConvertData.fah_to_cel(x.TAVG_lag_14d),
        PRECIPITATION_lag_7d=lambda x: ConvertData.inches_to_metric(inches=x.PRECIPITATION_lag_7d, unit="cm"),
        PRECIPITATION_lag_14d=lambda x: ConvertData.inches_to_metric(inches=x.PRECIPITATION_lag_14d, unit="cm"),
        Temp_Diff=lambda x: ConvertData.fah_to_cel(x.Temp_Diff),
        PPTWT=lambda x: ConvertData.inches_to_metric(inches=x.PPTWT, unit="mm")
    )
    .query("Snow_Density <= 0.6 and Snow_Density >= 0.05")
    .drop(
        columns=[
            'PRECIPITATION',
            'winter_start',
            'winter_end',
            'winter'
        ]
    )
)
clean_df2.info()
clean_df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2722562 entries, 0 to 6875222
Data columns (total 17 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   Date                   datetime64[ns, UTC]
 1   Site                   object             
 2   SWE                    float64            
 3   Elevation              float64            
 4   Snow_Class             object             
 5   Station_Name           object             
 6   Snow_Depth             float64            
 7   Sturm_DOWY             float64            
 8   Pistochi_DOWY          int64              
 9   DOWY                   int64              
 10  Temp_Diff              float64            
 11  TAVG_lag_7d            float64            
 12  TAVG_lag_14d           float64            
 13  PRECIPITATION_lag_7d   float64            
 14  PRECIPITATION_lag_14d  float64            
 15  PPTWT                  float64            
 16  Snow_Density           

,Date,Site,SWE,Elevation,Snow_Class,Station_Name,Snow_Depth,Sturm_DOWY,Pistochi_DOWY,DOWY,Temp_Diff,TAVG_lag_7d,TAVG_lag_14d,PRECIPITATION_lag_7d,PRECIPITATION_lag_14d,PPTWT,Snow_Density
0,2000-01-01 08:00:00+00:00,301:CA:SNTL,9.398,1886.712,Prairie,Adin Mtn,30.48,1.0,62,93,-0.103584,NaN,NaN,NaN,NaN,NaN,0.308333
1,2000-01-02 08:00:00+00:00,301:CA:SNTL,9.398,1886.712,Prairie,Adin Mtn,35.56,2.0,63,94,-0.103584,NaN,NaN,NaN,NaN,NaN,0.264286
2,2000-01-03 08:00:00+00:00,301:CA:SNTL,9.906,1886.712,Prairie,Adin Mtn,33.02,3.0,64,95,-0.103584,NaN,NaN,NaN,NaN,NaN,0.300000
3,2000-01-04 08:00:00+00:00,301:CA:SNTL,9.906,1886.712,Prairie,Adin Mtn,33.02,4.0,65,96,-0.103584,NaN,NaN,NaN,NaN,NaN,0.300000
4,2000-01-05 08:00:00+00:00,301:CA:SNTL,9.906,1886.712,Prairie,Adin Mtn,35.56,5.0,66,97,-0.103584,NaN,NaN,NaN,NaN,NaN,0.278571


In [9]:
clean_df2.describe().T

,count,mean,std,min,25%,50%,75%,max
SWE,2722562.0,30.827936,28.205947,3.048000,11.430000,22.352000,40.640000,310.134000
Elevation,2722562.0,2277.863989,720.333122,48.768000,1807.464000,2412.492000,2810.256000,3543.604800
Snow_Depth,2722562.0,95.798274,68.500037,5.080000,45.720000,81.280000,127.000000,711.200000
Sturm_DOWY,2718354.0,44.314528,57.097875,-92.000000,-2.000000,44.000000,90.000000,182.000000
Pistochi_DOWY,2722562.0,111.630646,63.376345,1.000000,62.000000,108.000000,154.000000,366.000000
DOWY,2722562.0,136.840510,57.140983,1.000000,91.000000,136.000000,182.000000,366.000000
Temp_Diff,2689631.0,4.334877,11.844961,-17.777778,2.244803,4.273259,5.954480,1234.817542
TAVG_lag_7d,2658468.0,-19.074164,2.865083,-40.000000,-20.873016,-18.936508,-17.150794,4.000000
TAVG_lag_14d,2660020.0,-19.117683,2.602980,-39.722222,-20.813492,-19.023810,-17.341270,4.000000
PRECIPITATION_lag_7d,2715764.0,0.430211,0.497399,0.000000,0.108857,0.290286,0.580571,7.958667


* Check missing observations.

In [10]:
clean_df2.isnull().sum()

Date                         0
Site                         0
SWE                          0
Elevation                    0
Snow_Class                   0
Station_Name                 0
Snow_Depth                   0
Sturm_DOWY                4208
Pistochi_DOWY                0
DOWY                         0
Temp_Diff                32931
TAVG_lag_7d              64094
TAVG_lag_14d             62542
PRECIPITATION_lag_7d      6798
PRECIPITATION_lag_14d     7665
PPTWT                    65787
Snow_Density                 0
dtype: int64

The missing values come from **Temperature and precipitation metrics**. Hence, we will not drop them so that we don't loose useful data points.

In [11]:
clean_df2.Date.dt.month.value_counts().sort_index()

Date
1     463981
2     424558
3     455532
4     385157
5     227436
6      51534
7       2745
8         28
9       1435
10     43164
11    230081
12    436911
Name: count, dtype: int64

* Let's see the states with observations in July, August, and September

In [12]:
(clean_df2[clean_df2.Date.dt.month == 8]).Site.str.split(':').map(lambda x: x[1]).value_counts()

Site
WA    23
CA     4
MT     1
Name: count, dtype: int64

In [13]:
(clean_df2[clean_df2.Date.dt.month == 9]).Site.str.split(':').map(lambda x: x[1]).value_counts()

Site
WY    482
MT    461
CO    178
UT    107
ID    101
AK     41
OR     34
WA     15
NV     14
SD      1
CA      1
Name: count, dtype: int64

In [14]:
(clean_df2[clean_df2.Date.dt.month == 7]).Site.str.split(':').map(lambda x: x[1]).value_counts()

Site
MT    735
WA    689
WY    324
ID    261
CA    232
OR    165
CO    136
UT    102
AK     68
NV     33
Name: count, dtype: int64

## Store Data 

Let's store this data for later use.

In [15]:
clean_df2.to_parquet(path='../data/clean_data_for_analysis.parquet', compression='gzip') # save to parquet file